In [1]:
from numpy import inf
from itertools import groupby

In [2]:
path_to_csv = '/data/lsml/sga/clickstream.csv'

In [3]:
clickstream = sc.textFile(path_to_csv)

In [4]:
header = clickstream.first()

In [5]:
clickstream = clickstream.filter(lambda x: x != header)

In [6]:
clickstream = clickstream.map(lambda x: x.split('\t'))
clickstream = clickstream.map(lambda x: (x[0] + x[1], x[2], x[3], x[4]))

In [8]:
# get dict of error time for each user session 
stop_time = clickstream.map(lambda x: (x[0], int(x[3])) if 'error' in x[1] else (x[0], inf))
stop_time = stop_time.reduceByKey(min)
stop_time_dict = stop_time.collectAsMap()

In [9]:
# get column of error time for each user session
clickstream = clickstream.map(lambda x: (x[0], x[1], x[2], x[3], stop_time_dict[x[0]]))

In [10]:
# remove logs after error occured
clickstream = clickstream.filter(lambda x: int(x[3]) < x[4])

In [11]:
# get routes with duplicate pages
routs = clickstream.map(lambda x: (x[0], x[2])).reduceByKey(lambda a, b: a + '-' + b)

In [12]:
# remove diplicates from line like 'main-main-tarrif' -> 'main-tarrif'
def duplicate_remover(line):
    pages_list = line[1].split('-')
    pages_list_no_duplicates = [key for key, grp in groupby(pages_list)]
    pages_string = '-'.join(pages_list_no_duplicates)
    return pages_string

In [13]:
# remove duplicates
routs = routs.map(lambda x: (duplicate_remover(x), 1))

In [14]:
output = routs.reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1], ascending=False).take(30)

In [15]:
answer = ''
for line in output:
    answer += line[0] + '\t' + str(line[1]) + '\n'

In [16]:
with open("/home/aabilov/output.txt", "w") as text_file:
    text_file.write(answer)
    text_file.close()

In [17]:
! cat /home/aabilov/output.txt

main	39506
main-tariffs	6580
main-news	6314
main-archive	5886
main-family	4884
main-digital	4248
main-bonus	3505
main-tariffs-news	1196
main-news-tariffs	1141
main-tariffs-archive	1039
main-news-archive	1020
main-archive-tariffs	1004
main-archive-news	999
main-news-family	927
main-family-tariffs	923
main-tariffs-family	918
main-family-news	886
main-archive-family	822
main-news-digital	801
main-tariffs-main	789
main-family-archive	777
main-tariffs-digital	757
main-digital-news	752
main-digital-tariffs	729
main-archive-digital	719
main-spravka	708
main-news-main	700
main-digital-archive	683
main-tariffs-bonus	666
main-archive-main	624


In [18]:
! curl -d "$(cat /home/aabilov/output.txt)" hadoop2-00.yandex.ru:8008/sga/task_hive

Great job! Secret keyword is 'HiveMind'
